# SSEShutdownHandler

> Handles graceful shutdown of SSE connections when the server exits.

In [ ]:
#| default_exp shutdown

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from uvicorn.main import Server
import asyncio
import time
from datetime import datetime
from typing import Set, Optional

In [ ]:
#| export
class SSEShutdownHandler:
  """
  Handles graceful shutdown of SSE connections when the server exits, 
  by ensuring all SSE clients receive a shutdown notification before 
  the server terminates, allowing them to cleanly close connections 
  without attempting reconnection.
  """

  def __init__(self, sse_manager, shutdown_delay: float = 1.0):
      self.sse_manager = sse_manager
      self.shutdown_delay = shutdown_delay
      self.should_exit = False
      self.active_connections: Set = set()
      self.shutdown_event = asyncio.Event()
      self._original_handler = None

  def install(self):
      """Install the shutdown handler into Uvicorn's Server class."""
      self._original_handler = Server.handle_exit
      Server.handle_exit = self._handle_exit

  def uninstall(self):
      """Restore the original Uvicorn handler."""
      if self._original_handler:
          Server.handle_exit = self._original_handler

  def _handle_exit(self, *args, **kwargs):
      """Custom exit handler that notifies SSE clients before shutdown."""
      print("\nInitiating graceful SSE shutdown...")
      self.should_exit = True
      self.shutdown_event.set()

      # Send shutdown message directly to all SSE connection queues
      try:
          print(f"Notifying {self.sse_manager.connection_count} SSE connections...")

          shutdown_message = {
              "type": "shutdown",
              "timestamp": datetime.now().isoformat(),
              "data": {"message": "Server shutting down"}
          }

          # Send directly to all connection queues
          for queue in list(self.sse_manager.connections):
              try:
                  queue.put_nowait(shutdown_message)
              except asyncio.QueueFull:
                  print("Queue full, couldn't send shutdown message")
              except Exception as e:
                  print(f"Error sending shutdown to queue: {e}")

          print(f"Sent shutdown message to {len(self.sse_manager.connections)} connections")

          # Allow time for clients to process shutdown messages
          time.sleep(self.shutdown_delay)

      except Exception as e:
          print(f"Error during SSE shutdown: {e}")

      # Clean up active connections
      self._cleanup_connections()

      # Call original handler
      if self._original_handler:
          self._original_handler(*args, **kwargs)

  def _cleanup_connections(self):
      """Cancel any remaining active connections."""
      if self.active_connections:
          print(f"Closing {len(self.active_connections)} active connections...")
          for connection in list(self.active_connections):
              try:
                  connection.cancel()
              except Exception as e:
                  print(f"Error closing connection: {e}")
          self.active_connections.clear()

  def track_connection(self, task):
      """Track an active connection task."""
      self.active_connections.add(task)

  def untrack_connection(self, task):
      """Remove a connection task from tracking."""
      self.active_connections.discard(task)

In [ ]:
from cjm_fasthtml_sse.core import SSEBroadcastManager

# Initialize the SSE Broadcast Manager
sse_manager = SSEBroadcastManager(
    max_queue_size=100,
    history_size=50,
    default_timeout=0.1
)

# Initialize the SSEShutdownHandler
shutdown_handler = SSEShutdownHandler(sse_manager)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()